---
title: "Polars crashcourse (basic polars)"
date: "08/10/2023"
author: 
    name: "Rodrigo Zepeda-Tello"
title-block-banner: true
toc: true
number-sections: true
format: 
  html:
    fontsize: 1.1em
    code-fold: false
    code-tools: true
    page-layout: full
    body-width: 1200px
jupyter: python3
---

# Why polars?

There are three characteristics which, from my perspective, make `polars` a pretty cool library:

1. **Allows for data larger than RAM**: It helps manipulate datasets that are larger than `RAM` memory. This means that you can manipulate any data as long as it fits your hard-drive.

2. **Fast** A [quick comparison of `polars` vs `pandas`](https://www.datacamp.com/tutorial/high-performance-data-manipulation-in-python-pandas2-vs-polars) shows that `polars` is faster from [2x up to 46x times faster than `pandas`](https://www.pola.rs/benchmarks.html).

   a. One of the key reasons polars is faster is that it allows for `parallel` computation out-of-the -box.
   
   b. `Polars` allows the user to perform `lazy` evaluations (to be explained later) and thus optimizes queries.

You can get more info about `polars` [at their website](https://www.pola.rs/). 


# Preparing for the tutorial

## Installation 

### From pip

If you are using `pip` you can install via: 
```bash
pip install polars[all]
```

### From conda

If you are using `conda`, first activate your environment and then:
```bash
conda install -c conda-forge -y polars numpy pyarrow xlsx2csv pandas
```

If you are using Windows you might also need to install the `timezone` package.


## Data

For the purpose of the tutorial I'm using three different databases.

1. **covidmx.parquet** Mexico's COVID-19 cases open dataset. This database contains millions of registries that cannot be processed in using RAM memory only. The original data has been downloaded from [Mexico's Health Ministry](https://www.gob.mx/salud/documentos/datos-abiertos-152127). However, for the purpose of this tutorial I've already pre-formatted the data from the ministry.

2. **inegi_population.csv** Mexico's population by state, municipality and locality according to 2020's census. Data comes from Mexico's Institute of Statistics and Geography (INEGI). 

3. **penguins.csv** The classical [`palmerpenguins`](https://allisonhorst.github.io/palmerpenguins/) dataset. Contains information of penguins in Antartida collected from the Palmer Station. 

::: {.callout-important title="Data"}
Download the data from the [Open Science Framework](https://www.doi.org/10.17605/OSF.IO/B5U7K) and save it into your working directory.
:::



## Additional modules

Some of the modules we are using for this tutorial are:

1. **os** For setting file paths, directories and other operating-system dependent functions. 
2. **polars** Basic for the tutorial!
3. **pandas** To transform the data so that the jupyter notebook and the seaborn plots can use them
4. **seaborn** For creating nice plots.


In [1]:
## Loading modules
import os
import polars as pl
import pandas as pd
import seaborn as sns

# Reading data

## Basic reading

Data can be read from `csv` files (`read_csv`), `parquet` files[^1] (`read_parquet`), `Excel` files (`read_excel`) and databases from other sources such as `SQL` (`read_database`) and Delta Lakes (`read_delta`). See [the documentation](https://pola-rs.github.io/polars/py-polars/html/reference/io.html) for more database-input options. 

[^1]: Text files optimized for columnar data

One can read the penguins `csv` file:


In [32]:
penguins = pl.read_csv("penguins.csv", null_values = "NA") #Missing values in the csv are represented as "NA"

A quick description of the variables can be obtained with `glimpse`:

In [33]:
penguins.glimpse()

Rows: 344
Columns: 8
$ species           <str> Adelie, Adelie, Adelie, Adelie, Adelie, Adelie, Adelie, Adelie, Adelie, Adelie
$ island            <str> Torgersen, Torgersen, Torgersen, Torgersen, Torgersen, Torgersen, Torgersen, Torgersen, Torgersen, Torgersen
$ bill_length_mm    <f64> 39.1, 39.5, 40.3, None, 36.7, 39.3, 38.9, 39.2, 34.1, 42.0
$ bill_depth_mm     <f64> 18.7, 17.4, 18.0, None, 19.3, 20.6, 17.8, 19.6, 18.1, 20.2
$ flipper_length_mm <i64> 181, 186, 195, None, 193, 190, 181, 195, 193, 190
$ body_mass_g       <i64> 3750, 3800, 3250, None, 3450, 3650, 3625, 4675, 3475, 4250
$ sex               <str> male, female, female, None, female, male, female, male, None, None
$ year              <i64> 2007, 2007, 2007, 2007, 2007, 2007, 2007, 2007, 2007, 2007



One can obtain descriptors for the number of rows and number of columns with the `shape` attribute:

In [4]:
penguins.shape

(344, 8)

Equivalent attributes are the `width` (number of columns) and `height` (number of rows)

In [5]:
print("There are %d columns," % penguins.width)
print("and %d rows." % penguins.height)

There are 8 columns,
and 344 rows.


One can also take a look at the first `n=7` rows with `head`:

In [6]:
penguins.head(n=7)

species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
str,str,str,str,str,str,str,i64
"""Adelie""","""Torgersen""","""39.1""","""18.7""","""181""","""3750""","""male""",2007
"""Adelie""","""Torgersen""","""39.5""","""17.4""","""186""","""3800""","""female""",2007
"""Adelie""","""Torgersen""","""40.3""","""18""","""195""","""3250""","""female""",2007
"""Adelie""","""Torgersen""","""NA""","""NA""","""NA""","""NA""","""NA""",2007
"""Adelie""","""Torgersen""","""36.7""","""19.3""","""193""","""3450""","""female""",2007
"""Adelie""","""Torgersen""","""39.3""","""20.6""","""190""","""3650""","""male""",2007
"""Adelie""","""Torgersen""","""38.9""","""17.8""","""181""","""3625""","""female""",2007


The last `n=3` rows of the dataset can be displayed with `tail`:

In [7]:
penguins.tail(n=3)

species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
str,str,str,str,str,str,str,i64
"""Chinstrap""","""Dream""","""49.6""","""18.2""","""193""","""3775""","""male""",2009
"""Chinstrap""","""Dream""","""50.8""","""19""","""210""","""4100""","""male""",2009
"""Chinstrap""","""Dream""","""50.2""","""18.7""","""198""","""3775""","""female""",2009


By default, jupyter notebooks only allow us to display a certain amount of rows (`display.max_rows`), columns (`display.max_columns`) and with a certain width (`display.max_colwidth`). This can be changed globally by putting `pd.option_context` at the start of the script or just for one print with a `with` (which creates a [context](https://book.pythontips.com/en/latest/context_managers.html)). As jupyter is designed for `pandas` one needs to convert the polars `DataFrame` into a `pandas` one with `to_pandas()`. For example, here we are only showing the first `3` columns (including the row number) and `12` rows: 

In [17]:
with pd.option_context('display.max_rows', 12, 'display.max_columns', 3,'display.max_colwidth', None):
    display(penguins.to_pandas().head(n=12)) #Notice that head must be specified here

,species,...,year
0,Adelie,...,2007
1,Adelie,...,2007
2,Adelie,...,2007
3,Adelie,...,2007
4,Adelie,...,2007
5,Adelie,...,2007
6,Adelie,...,2007
7,Adelie,...,2007
8,Adelie,...,2007
9,Adelie,...,2007


The `sample` command obtains a random sample (without replacement) of the rows of size `n` and keeps it. This command is ideal for debugging `polars` programs as it allows the user to create a smaller database of which one can keep a track of. For example, the following obtains a random sample of size `9` of the data: 

In [20]:
penguins.sample(n = 9)

species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
str,str,str,str,str,str,str,i64
"""Adelie""","""Dream""","""39.2""","""18.6""","""190""","""4250""","""male""",2009
"""Gentoo""","""Biscoe""","""47.7""","""15""","""216""","""4750""","""female""",2008
"""Gentoo""","""Biscoe""","""47.6""","""14.5""","""215""","""5400""","""male""",2007
"""Gentoo""","""Biscoe""","""50.8""","""15.7""","""226""","""5200""","""male""",2009
"""Gentoo""","""Biscoe""","""52.5""","""15.6""","""221""","""5450""","""male""",2009
"""Gentoo""","""Biscoe""","""50.5""","""15.9""","""222""","""5550""","""male""",2008
"""Adelie""","""Biscoe""","""40.6""","""18.6""","""183""","""3550""","""male""",2007
"""Gentoo""","""Biscoe""","""49.6""","""15""","""216""","""4750""","""male""",2008
"""Adelie""","""Torgersen""","""39""","""17.1""","""191""","""3050""","""female""",2009


To obtain an array of the column names one can do:

In [38]:
penguins.columns

['species',
 'island',
 'bill_length_mm',
 'bill_depth_mm',
 'flipper_length_mm',
 'body_mass_g',
 'sex',
 'year']

## Advanced reading (scanning)

Larger datasets benefit from `scanning`. When a file is `scanned` it is not read. `polars` only registers that it `will`read it. To actually read it you need to perform a `collect` after a scan. This is useful as `polars` optimizes all the queries you produce into the scan so that when you `collect` you obtain a `faster` result than performing the operations one by one. 

Let's see an example by reading the `covidmx` dataset and removing all rows except for those ones that occured in Mexico City which corresponds to `ENTIDAD_UM == "09"`. 

In [24]:
%timeit -n1 -r1 pl.read_parquet("covidmx.parquet").filter(pl.col("ENTIDAD_UM") == "09")

20.4 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


The previous line `reads` the complete file _and then_  filters by ensuring to keep only the values where the column `ENTIDAD_UM` equals `"09"`. If we use `scan` its way faster as it will collapses the `scan` and the `filter` into one operation and reads only the columns that have a `09` value

In [25]:
%timeit -n1 -r1 pl.scan_parquet("covidmx.parquet").filter(pl.col("ENTIDAD_UM") == "09").collect()

10.2 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


This idea of not conducting all operations till all the queries are given is called **[lazy evaluation](https://en.wikipedia.org/wiki/Lazy_evaluation)** and is one of the keys for `polars`' speed. 

## Exercise

1. Read the `inegi_population.csv` file,
2. Obtain a random sample of `n = 50` rows,
3. Display the complete random sample (all rows and columns) in the jupyter notebook.

# Thinking polars

By design, `polars` data should be in a [tidy](https://doi.org/10.18637/jss.v059.i10) format with rows corresponding to observations and columns corresponding to variables measured from those observations.

![Image of a tidy dataframe showing that columns are variables and rows observations](df_images/column_row_format.svg){width=80%}

Polars is optimized, by design, for columnar data; in particular for its storage. Let's talk about it for a second. 

## SIMD

Let's think of the data for a second. Consider the first five entries of the following subset of the data:

In [37]:
penguins.select(pl.col("bill_length_mm","sex","year")).head()

bill_length_mm,sex,year
f64,str,i64
39.1,"""male""",2007
39.5,"""female""",2007
40.3,"""female""",2007
null,null,2007
36.7,"""female""",2007


The first one, `bill_length_mm` is a float (`f64`) which allows for decimal numbers, the second one, `sex` is a string (`str`) while the last one, `year` is an integer (`i64`). Each of these columns have types (`dtypes` in Python) that allow for different operations. For example, one can sum `+1` to both `bill_length_mm` and `year` but not to `sex`. 

The traditional way to represents data in memory is rowwise which means that the first entry of the float `bill_length_mm` row is next-to (in memory) the string `sex` whose memory address is next to the integer `year`. That means that performing operations in one column (say summing `+1` to year) involves jumping through different memory addresses. 

![Image of a `DataFrame` being stored into memory with two different formats rowwise and columnwise](df_images/dataframe_storage.svg){width=50%}

The columnar format of data stores one column in contiguous memory addresses, then the next column and then the next one. Modern processors are equipped with a gizmo called [**single instruction, multiple data (SIMD)**](https://en.wikipedia.org/wiki/Single_instruction,_multiple_data). SIMD allows for instructions to be performed in memory blocks without specifying the specific address of each member of the block. Thus, storing the columns next to each other generates memory blocks into which SIMD can be applied. 

While, for summing `+1` in the classical `rowwise` data storage one has to go specifying the memory address of the first year, then the address of the second and so on; in the `columar` approach one has only to specify the year memory block. 


## Why should I care?

Polars is designed with the columnar format in mind. That means that to conduct operations in your `DataFrame` one has to think the operations as operations **for the whole column**. This doesn't mean that `rowwise` operations cannot be done. However, `rowwise` operations (as well as `for` loops) are usually suboptimal for `polars` `DataFrames`.  

# DataFrame expressions

In this section, we discuss several transformations one can apply into the `DataFrame`. 

## How to keep/remove stuff

### Select

![`select` chooses which columns to keep or drop](df_images/select.svg){width=75%}

The `select` command allows users to choose which columns to keep. This can be done by specifying the columns:

In [41]:
penguins.select(pl.col("sex","species")).head()

sex,species
str,str
"""male""","""Adelie"""
"""female""","""Adelie"""
"""female""","""Adelie"""
null,"""Adelie"""
"""female""","""Adelie"""


::: {.callout-tip title="Format"}
Code is usually formatted rowwise to avoid run-ons so that each instruction has its row
:::


In [42]:
(penguins
 .select(pl.col("sex","species"))
 .head())

sex,species
str,str
"""male""","""Adelie"""
"""female""","""Adelie"""
"""female""","""Adelie"""
null,"""Adelie"""
"""female""","""Adelie"""


One can select all columns with `.all`:

In [43]:
(penguins
 .select(pl.all())
 .head())

species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
str,str,f64,f64,i64,i64,str,i64
"""Adelie""","""Torgersen""",39.1,18.7,181,3750,"""male""",2007
"""Adelie""","""Torgersen""",39.5,17.4,186,3800,"""female""",2007
"""Adelie""","""Torgersen""",40.3,18.0,195,3250,"""female""",2007
"""Adelie""","""Torgersen""",null,null,null,null,null,2007
"""Adelie""","""Torgersen""",36.7,19.3,193,3450,"""female""",2007


The `all` command can be combined with the `exclude` to remove specific columns:

In [44]:
(penguins
 .select(pl.all().exclude("island"))
 .head())

species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
str,f64,f64,i64,i64,str,i64
"""Adelie""",39.1,18.7,181,3750,"""male""",2007
"""Adelie""",39.5,17.4,186,3800,"""female""",2007
"""Adelie""",40.3,18.0,195,3250,"""female""",2007
"""Adelie""",null,null,null,null,null,2007
"""Adelie""",36.7,19.3,193,3450,"""female""",2007


The previous code is equivalent to using the `drop` function. Note that by design `drop` doesn't use `pl.col` while `select` does. 

In [80]:
(penguins
 .drop("island")
 .head())

species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
str,f64,f64,i64,i64,str,i64
"""Adelie""",39.1,18.7,181,3750,"""male""",2007
"""Adelie""",39.5,17.4,186,3800,"""female""",2007
"""Adelie""",40.3,18.0,195,3250,"""female""",2007
"""Adelie""",null,null,null,null,null,2007
"""Adelie""",36.7,19.3,193,3450,"""female""",2007


[Regular expressions](https://learn.microsoft.com/en-us/dotnet/standard/base-types/regular-expression-language-quick-reference) can be used to select or exclude specific columns matching the expression. For example, to keep those that end up in `mm`:

In [52]:
import polars.selectors as cs

(penguins
 .select(cs.matches("_mm"))
 .head())

bill_length_mm,bill_depth_mm,flipper_length_mm
f64,f64,i64
39.1,18.7,181
39.5,17.4,186
40.3,18.0,195
null,null,null
36.7,19.3,193


To exclude them one needs to negate the operation with `~`:


In [53]:
(penguins
 .select(~cs.matches("_mm"))
 .head())

species,island,body_mass_g,sex,year
str,str,i64,str,i64
"""Adelie""","""Torgersen""",3750,"""male""",2007
"""Adelie""","""Torgersen""",3800,"""female""",2007
"""Adelie""","""Torgersen""",3250,"""female""",2007
"""Adelie""","""Torgersen""",null,null,2007
"""Adelie""","""Torgersen""",3450,"""female""",2007


Boolean operations can be applied inside a `select` thus allowing to apply an **and**

In [54]:
(penguins
 .select(cs.matches("_mm") & cs.matches("length"))
 .head())

bill_length_mm,flipper_length_mm
f64,i64
39.1,181
39.5,186
40.3,195
null,null
36.7,193


or an **or**

In [57]:
(penguins
 .select(cs.matches("_mm") | cs.matches("island"))
 .head())

bill_length_mm,bill_depth_mm,flipper_length_mm,island
f64,f64,i64,str
39.1,18.7,181,"""Torgersen"""
39.5,17.4,186,"""Torgersen"""
40.3,18.0,195,"""Torgersen"""
null,null,null,"""Torgersen"""
36.7,19.3,193,"""Torgersen"""


Columns can also be selected by `dtype` with the `cs.integer()`, `cs.float()`, `cs.numeric()` (float and int), `cs.string()`, `cs.datetime()` as well as [additional selectors](https://pola-rs.github.io/polars/py-polars/html/reference/selectors.html#functions) specified in the manual

In [61]:
#Select all numeric except year. The - acts as a set difference operator.
(penguins
 .select(cs.numeric() - cs.matches("year")) 
 .head())

bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
f64,f64,i64,i64
39.1,18.7,181,3750
39.5,17.4,186,3800
40.3,18.0,195,3250
null,null,null,null
36.7,19.3,193,3450


### Filter

![`filter` chooses which rows to keep or drop](df_images/filter.svg){width=75%}

The `filter` command allows users to choose which rows to keep based upon some `boolean` conditions of the columns. For example one can select those penguins which have a `body_mass_g` greater than `3500`

In [64]:
(penguins
 .filter(pl.col("body_mass_g") >= 3500)
 .head())

species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
str,str,f64,f64,i64,i64,str,i64
"""Adelie""","""Torgersen""",39.1,18.7,181,3750,"""male""",2007
"""Adelie""","""Torgersen""",39.5,17.4,186,3800,"""female""",2007
"""Adelie""","""Torgersen""",39.3,20.6,190,3650,"""male""",2007
"""Adelie""","""Torgersen""",38.9,17.8,181,3625,"""female""",2007
"""Adelie""","""Torgersen""",39.2,19.6,195,4675,"""male""",2007


More complicated filters can be obtained by combining the logical and `&`, or `|`, not `~` and difference `-` operators:

In [70]:
# Keep only Gentoo penguins with bills larger than 50 or flippers smaller than 170. 
(penguins
 .filter((pl.col("species") == "Gentoo") & ((pl.col("flipper_length_mm") < 170) | (pl.col("bill_length_mm") > 50)))
 .head())

species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
str,str,f64,f64,i64,i64,str,i64
"""Gentoo""","""Biscoe""",50.2,14.3,218,5700,"""male""",2007
"""Gentoo""","""Biscoe""",59.6,17.0,230,6050,"""male""",2007
"""Gentoo""","""Biscoe""",50.5,15.9,222,5550,"""male""",2008
"""Gentoo""","""Biscoe""",50.5,15.9,225,5400,"""male""",2008
"""Gentoo""","""Biscoe""",50.1,15.0,225,5000,"""male""",2008


### Special selectors

#### take_every
returns every `nth` row of the DataFrame

![`take_every` chooses every nth row](df_images/take_every.svg){width=75%}

In [72]:
#Return every third penguin
(penguins
 .take_every(n=3)
 .head())

species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
str,str,f64,f64,i64,i64,str,i64
"""Adelie""","""Torgersen""",39.1,18.7,181,3750,"""male""",2007
"""Adelie""","""Torgersen""",null,null,null,null,null,2007
"""Adelie""","""Torgersen""",38.9,17.8,181,3625,"""female""",2007
"""Adelie""","""Torgersen""",42.0,20.2,190,4250,null,2007
"""Adelie""","""Torgersen""",41.1,17.6,182,3200,"""female""",2007


#### `top_k` 
Returns the first `k` values of a column either by ascending or descending order

![`top_k` chooses every nth row](df_images/top_k.svg){width=75%}

In [75]:
#Return the top 3 fatter penguins:
(penguins
 .top_k(k=5, by = "body_mass_g", descending=True, nulls_last=True) 
 .head())

species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
str,str,f64,f64,i64,i64,str,i64
"""Chinstrap""","""Dream""",46.9,16.6,192,2700,"""female""",2008
"""Adelie""","""Biscoe""",36.5,16.6,181,2850,"""female""",2008
"""Adelie""","""Biscoe""",36.4,17.1,184,2850,"""female""",2008
"""Adelie""","""Torgersen""",38.6,17.0,188,2900,"""female""",2009
"""Adelie""","""Dream""",33.1,16.1,178,2900,"""female""",2008


#### `drop_null` 
Removes all rows that have missing values in a certain column (leave blank for all columns)

![`drop_null` removes all null rows that have null values from a column (or all columns)](df_images/drop_null.svg){width=75%}

In [85]:
#Compare against the take_every 3 previously specified
(penguins
 .take_every(n=3)
 .drop_nulls(["bill_length_mm"])
 .head())

species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
str,str,f64,f64,i64,i64,str,i64
"""Adelie""","""Torgersen""",39.1,18.7,181,3750,"""male""",2007
"""Adelie""","""Torgersen""",38.9,17.8,181,3625,"""female""",2007
"""Adelie""","""Torgersen""",42.0,20.2,190,4250,null,2007
"""Adelie""","""Torgersen""",41.1,17.6,182,3200,"""female""",2007
"""Adelie""","""Torgersen""",36.6,17.8,185,3700,"""female""",2007


### Exercise

Answer the following questions:

1. How many Adelie penguins are from Biscoe Island?
   
2. What is the proportion of male penguins in the database?
   
3. What's the probability that a penguin with flippers less than `200` mm has a body mass greater than 5000?

4. Which species has the penguins with the largest bills?

## How to 